# SQL Murder Mystery

In this notebook we will be solving the [SQL Murder Mystery](https://mystery.knightlab.com) designed by [Knight Lab](https://knightlab.northwestern.edu), a fun little game where we have to solve the crime by querying the database.

To begin with we will connect to the database. The %sql magic allows us to issue SQL commands within this notebook.

In [1]:
%load_ext sql

In [2]:
%sql sqlite:///sql-murder-mystery.db

## Prompt
A prompt is provided to get us started on solving this mystery.

>A crime has taken place and the detective needs your help. The detective gave you the 
crime scene report, but you somehow lost it. You vaguely remember that the crime 
was a **murder** that occurred sometime on **Jan.15, 2018** and that it took place in **SQL 
City**. Start by retrieving the corresponding crime scene report from the police 
department’s database.

In [3]:
%sql SELECT name \
FROM sqlite_master \
WHERE type = 'table'

 * sqlite:///sql-murder-mystery.db
Done.


name
crime_scene_report
drivers_license
person
facebook_event_checkin
interview
get_fit_now_member
get_fit_now_check_in
income
solution


The above shows the different tables available in the database. According to the prompt, crime_scene_report should be the first table we explore.

## Crime Scene Report

In [4]:
%sql SELECT sql \
FROM sqlite_master \
WHERE name = 'crime_scene_report'

 * sqlite:///sql-murder-mystery.db
Done.


sql
"CREATE TABLE crime_scene_report ( date integer, type text, description text, city text )"


This query will tell us the column names and the corresponding data types inside the table, allowing us to infer the table and make additional queries.

In [5]:
%sql SELECT * \
FROM crime_scene_report \
WHERE type = 'murder' \
AND date BETWEEN 20180101 AND 20180131 \
AND lower(city) = 'sql city'

 * sqlite:///sql-murder-mystery.db
Done.


date,type,description,city
20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


Inserting the conditions found in the prompt, we find only one crime scene report that matches. The description mentions two witnesses which we will now find.

## Witness 1

In [6]:
%sql SELECT sql \
FROM sqlite_master \
WHERE name = 'person'

 * sqlite:///sql-murder-mystery.db
Done.


sql
"CREATE TABLE person ( id integer PRIMARY KEY, name text, license_id integer, address_number integer, address_street_name text, ssn integer, FOREIGN KEY (license_id) REFERENCES drivers_license(id) )"


In [7]:
%sql SELECT * \
FROM person \
WHERE lower(address_street_name) = 'northwestern dr'\
ORDER BY address_number DESC \
LIMIT 1

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn
14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


The first witness living in the last house on Northwestern Dr is Morty Schapiro.

## Witness 2

In [8]:
%sql SELECT * \
FROM person \
WHERE lower(address_street_name) = 'franklin ave' \
AND lower(name) LIKE '%annabel%'

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn
16371,Annabel Miller,490173,103,Franklin Ave,318771143


The second witness living on Franklin Ave is Annabel Miller.

## Interview Transcripts

In [9]:
%sql SELECT sql \
FROM sqlite_master \
WHERE name = 'interview'

 * sqlite:///sql-murder-mystery.db
Done.


sql
"CREATE TABLE interview ( person_id integer, transcript text, FOREIGN KEY (person_id) REFERENCES person(id) )"


Looking at the structure of the 'interview' table we can see that 'person_id' column in 'interview' references 'id' columns from 'person', which means that we can use those columns as common identifiers of the two tables.

In [10]:
%sql SELECT * \
FROM interview \
JOIN person \
ON interview.person_id = person.id \
WHERE interview.person_id = 16371 OR interview.person_id = 14887

 * sqlite:///sql-murder-mystery.db
Done.


person_id,transcript,id,name,license_id,address_number,address_street_name,ssn
14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W"".",14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.",16371,Annabel Miller,490173,103,Franklin Ave,318771143


By simply using the ids of Morty and Annabel we can find their interview transcripts detailing their witness reports. Both transcripts point us to the "Get Fit Now" gym.

## Get Fit Now Members

In [11]:
%sql SELECT sql \
FROM sqlite_master \
WHERE name = 'get_fit_now_member'

 * sqlite:///sql-murder-mystery.db
Done.


sql
"CREATE TABLE get_fit_now_member ( id text PRIMARY KEY, person_id integer, name text, membership_start_date integer, membership_status text, FOREIGN KEY (person_id) REFERENCES person(id) )"


In [12]:
%sql SELECT * \
FROM get_fit_now_member \
WHERE lower(membership_status) = 'gold' \
AND id LIKE '48Z%'

 * sqlite:///sql-murder-mystery.db
Done.


id,person_id,name,membership_start_date,membership_status
48Z7A,28819,Joe Germuska,20160305,gold
48Z55,67318,Jeremy Bowers,20160101,gold


Running the query on the table containing gym membership information we find two people matching the conditions of having a 'gold' membership status and membership id starting with '48Z' from Morty's witness report.

## Get Fit Now Checkin

In [13]:
%sql SELECT sql \
FROM sqlite_master \
WHERE name = 'get_fit_now_check_in'

 * sqlite:///sql-murder-mystery.db
Done.


sql
"CREATE TABLE get_fit_now_check_in ( membership_id text, check_in_date integer, check_in_time integer, check_out_time integer, FOREIGN KEY (membership_id) REFERENCES get_fit_now_member(id) )"


In [14]:
%sql SELECT * \
FROM get_fit_now_check_in check_in \
JOIN get_fit_now_member member \
ON check_in.membership_id = member.id \
WHERE person_id = 16371

 * sqlite:///sql-murder-mystery.db
Done.


membership_id,check_in_date,check_in_time,check_out_time,id,person_id,name,membership_start_date,membership_status
90081,20180109,1600,1700,90081,16371,Annabel Miller,20160208,gold


Verifying Annabel's witness report we find out that on 9 January she was in the gym from 1600 to 1700, indicating that the murderer was in the gym during that period of time as well.

In [15]:
%sql SELECT * \
FROM get_fit_now_check_in \
WHERE check_in_date = 20180109 \
AND membership_id LIKE '48Z%'

 * sqlite:///sql-murder-mystery.db
Done.


membership_id,check_in_date,check_in_time,check_out_time
48Z7A,20180109,1600,1730
48Z55,20180109,1530,1700


Querying the table for members who checked in to the gym on 9 January and have membership ids starting with '48Z' results in the same two people from before. Additionally, both of them we in the gym when Annabel was present so it is not possible to identify the killer.

## Driver's License
Fortunately, Morty's witness report details the killer entering a vehicle with car plate number containing "H42W".

In [16]:
%sql SELECT sql \
FROM sqlite_master \
WHERE name = 'drivers_license'

 * sqlite:///sql-murder-mystery.db
Done.


sql
"CREATE TABLE drivers_license ( id integer PRIMARY KEY, age integer, height integer, eye_color text, hair_color text, gender text, plate_number text, car_make text, car_model text )"


In [17]:
%sql SELECT * \
FROM drivers_license \
JOIN person \
ON drivers_license.id = person.license_id \
WHERE person.id = 28819 OR person.id = 67318

 * sqlite:///sql-murder-mystery.db
Done.


id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id_1,name,license_id,address_number,address_street_name,ssn
423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


Querying the driver's license table with ids of the two suspects returns only one result from Jeremy Bowers with car plate number matching the description, suggesting that he was the killer who entered his own registered vehicle after the murder.

## Verify Answer

In [18]:
%sql INSERT INTO solution VALUES (1, 'Jeremy Bowers');
%sql SELECT value FROM solution;

 * sqlite:///sql-murder-mystery.db
1 rows affected.
 * sqlite:///sql-murder-mystery.db
Done.


value
"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."


It seems that we have indeed identified the correct killer! Now a bonus challenge requires us to identify the mastermind behind this murder.

In [19]:
%sql SELECT * \
FROM interview \
WHERE person_id = 67318

 * sqlite:///sql-murder-mystery.db
Done.


person_id,transcript
67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017."


Looking into the interview transcript of the murder we gather some identifying features of this mastermind.

In [20]:
%sql SELECT * \
FROM person \
JOIN drivers_license dl \
ON person.license_id = dl.id \
JOIN facebook_event_checkin fb \
ON fb.person_id = person.id \
JOIN income \
ON income.ssn = person.ssn \
WHERE height BETWEEN 65 AND 67 \
AND lower(hair_color) = 'red' \
AND car_make = 'Tesla' \
AND car_model = 'Model S'

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn,id_1,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,person_id,event_id,event_name,date,ssn_1,annual_income
99716,Miranda Priestly,202298,1883,Golden Ave,987756388,202298,68,66,green,red,female,500123,Tesla,Model S,99716,1143,SQL Symphony Concert,20171206,987756388,310000
99716,Miranda Priestly,202298,1883,Golden Ave,987756388,202298,68,66,green,red,female,500123,Tesla,Model S,99716,1143,SQL Symphony Concert,20171212,987756388,310000
99716,Miranda Priestly,202298,1883,Golden Ave,987756388,202298,68,66,green,red,female,500123,Tesla,Model S,99716,1143,SQL Symphony Concert,20171229,987756388,310000


Simply by using the height, hair color, and car make/model we were able to narrow the search down to only one suspect Miranda Priestly. Additionally, from the table we observe that she has indeed attended the SQL Symphony Concert thrice in December 2017 and makes over 300K annually, matching the murderer's interview transcript.

In [21]:
%sql INSERT INTO solution VALUES (1, 'Miranda Priestly');
%sql SELECT value FROM solution;

 * sqlite:///sql-murder-mystery.db
1 rows affected.
 * sqlite:///sql-murder-mystery.db
Done.


value
"Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!"


And as suspected, Miranda is the mastermind behind this murder!